# Experiment 1

### Description of the experiment...

We are going to reply the experiment

Talk about How the paper "Application of Deep Reinforcement Learning on Automated Stock Trading" writed by Lin Chen and Qiang Gao in 2019 apply DQN to trading S&P 500 ETF stocks Daily rate 

Data from 01/01/2000 to 12/31/2018

Training period:
(01/01/2000-12/31/2004)

Testing period:
(01/01/2005-12/31/2018)

Reward:
the reward is calculated by the difference between next day’s adjusted close and the current day’s adjusted close

(Im gonna add cach and stocks_hold and total price of portfolio just to have a evaluation metric)

Basicly what this model do is try to predict if the prediction of NN is write at buy or sell

NN Architect:

- Input: 22 Nodes (Features)
- Hidden Layer 1: 10 Nodes (ReLU)
- Hidden Layer 2: 10 Nodes (ReLU)
- Output: 3 Nodes (Actions)

Input:
- **Last 20 prices of stocks**: 20 Nodes with the price of the last 20 prices
- **Cash**: Actual Cash to buy a stock
- **Shares_held**: Number of S&P 500 ETF shares currently owned by the agent. 

Output:
- **Hold:** Do nothing, maintain current position.
- **Buy:** Buy 1 stock.
- **Sell:** Sell 1 stock.

### Get S&P 500 ETF Stocks Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Read DataFrame
SPY_stocks = pd.read_csv("../data/Preprocessed/SPY_Data.csv")
SPY_stocks = SPY_stocks.drop(['Unnamed: 0', 'High', 'Low'], axis=1)
SPY_stocks.head()

,Date,Close
0,1999-07-26,84.923210
1,1999-07-27,85.632195
2,1999-07-28,85.907906
3,1999-07-29,84.706596
4,1999-07-30,83.662720


### Generate Input Parameters of NN

In [6]:
inputs_SPY = SPY_stocks.copy()

window_size = 20 

for i in range(1, window_size):
    inputs_SPY[f'Close_t-{i}'] = inputs_SPY['Close'].shift(i)

In [8]:
inputs_SPY.head(5)

,Date,Close,Close_t-1,Close_t-2,Close_t-3,Close_t-4,Close_t-5,Close_t-6,Close_t-7,Close_t-8,...,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,Close_t-18,Close_t-19
0,1999-07-26,84.923210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-07-27,85.632195,84.923210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-07-28,85.907906,85.632195,84.923210,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1999-07-29,84.706596,85.907906,85.632195,84.923210,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-07-30,83.662720,84.706596,85.907906,85.632195,84.92321,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
inputs_SPY.shape

(6534, 21)

### Get years 2000-2004 to training and 2005-2018 to Testing

In [16]:
training_SPY = inputs_SPY.copy()

training_SPY = training_SPY[training_SPY['Date'] >= '2000-01-01']
training_SPY = training_SPY[training_SPY['Date'] <= '2005-01-01']
training_SPY.head()

,Date,Close,Close_t-1,Close_t-2,Close_t-3,Close_t-4,Close_t-5,Close_t-6,Close_t-7,Close_t-8,...,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,Close_t-18,Close_t-19
112,2000-01-03,92.142548,93.053238,92.904747,93.013657,92.617676,92.677139,92.805771,91.350555,91.113052,...,90.400291,89.823433,89.428421,88.954414,89.823433,89.665375,89.369156,88.934631,89.507401,90.238129
113,2000-01-04,88.539215,92.142548,93.053238,92.904747,93.013657,92.617676,92.677139,92.805771,91.350555,...,89.746895,90.400291,89.823433,89.428421,88.954414,89.823433,89.665375,89.369156,88.934631,89.507401
114,2000-01-05,88.697601,88.539215,92.142548,93.053238,92.904747,93.013657,92.617676,92.677139,92.805771,...,91.113052,89.746895,90.400291,89.823433,89.428421,88.954414,89.823433,89.665375,89.369156,88.934631
115,2000-01-06,87.272079,88.697601,88.539215,92.142548,93.053238,92.904747,93.013657,92.617676,92.677139,...,91.350555,91.113052,89.746895,90.400291,89.823433,89.428421,88.954414,89.823433,89.665375,89.369156
116,2000-01-07,92.340523,87.272079,88.697601,88.539215,92.142548,93.053238,92.904747,93.013657,92.617676,...,92.805771,91.350555,91.113052,89.746895,90.400291,89.823433,89.428421,88.954414,89.823433,89.665375


In [17]:
print("Oldst Date: ", training_SPY['Date'].min())
print("Recent Date: ", training_SPY['Date'].max())

Oldst Date:  2000-01-03
Recent Date:  2004-12-31


In [18]:
testing_SPY = inputs_SPY.copy()

testing_SPY = testing_SPY[testing_SPY['Date'] >= '2005-01-01']
testing_SPY = testing_SPY[testing_SPY['Date'] <= '2019-01-01']
testing_SPY.head()

,Date,Close,Close_t-1,Close_t-2,Close_t-3,Close_t-4,Close_t-5,Close_t-6,Close_t-7,Close_t-8,...,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,Close_t-18,Close_t-19
1368,2005-01-03,82.073990,82.462929,82.640297,82.797234,82.674400,82.224159,82.394691,82.333298,82.135445,...,81.487335,82.034492,82.081993,82.020889,81.735710,81.029503,80.948029,80.662827,80.194290,80.948029
1369,2005-01-04,81.071144,82.073990,82.462929,82.640297,82.797234,82.674400,82.224159,82.394691,82.333298,...,81.507767,81.487335,82.034492,82.081993,82.020889,81.735710,81.029503,80.948029,80.662827,80.194290
1370,2005-01-05,80.511719,81.071144,82.073990,82.462929,82.640297,82.797234,82.674400,82.224159,82.394691,...,82.135445,81.507767,81.487335,82.034492,82.081993,82.020889,81.735710,81.029503,80.948029,80.662827
1371,2005-01-06,80.921066,80.511719,81.071144,82.073990,82.462929,82.640297,82.797234,82.674400,82.224159,...,82.333298,82.135445,81.507767,81.487335,82.034492,82.081993,82.020889,81.735710,81.029503,80.948029
1372,2005-01-07,80.805069,80.921066,80.511719,81.071144,82.073990,82.462929,82.640297,82.797234,82.674400,...,82.394691,82.333298,82.135445,81.507767,81.487335,82.034492,82.081993,82.020889,81.735710,81.029503


In [19]:
print("Oldst Date: ", testing_SPY['Date'].min())
print("Recent Date: ", testing_SPY['Date'].max())

Oldst Date:  2005-01-03
Recent Date:  2018-12-31


### Creating the architecture of the Neural Network

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

In [22]:
class DQN(nn.Module):
    def __init__(self, input_dim=12, output_dim=7):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 10) #self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(10, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

### Create Custom Trading Environment